Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, welch
from scipy import stats
import mne
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score

Summary file parsing to extract seizure start and end times

In [ ]:
def extract_seizure_events_from_txt(data_path):
    """"
    Searches for the .txt summary file in a folder, parses seizure times.
    """
    seizure_info = {}
    summary_file = None

    for fname in os.listdir(data_path):
        if fname.endswith(".txt"):
            summary_file = os.path.join(data_path, fname)
            break
    if summary_file is None:
        raise FileNotFoundError("No Summary Text File Found!")
    
    current_file = None 
    with open(summary_file, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("File Name:"):
                current_file = line.split(":")[1].strip()
                seizure_info[current_file] = []
            elif line.startswith("Seizure Start Time:"):
                start_time = int(line.split(":")[1].strip().split()[0])
            elif line.startswith("Seizure End Time:"):
                end_time = int(line.split(":")[1].strip()[0])
                seizure_info[current_file].append((start_time, end_time))
    return seizure_info

Data Loading